In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from statistics import mean

import requests

Data for evaluating careers of NFL WR's:

NFL stats (2012-2022)

In [15]:
le_data = pd.read_csv('Data/yearly_stats.csv', dtype={'height': str}, parse_dates= False)
#issue as orignally I thought pandas was interpreting height as a date, but the csv is reporting heights as dates

#filter for WR postion since that is all we care about

WR_data = le_data[le_data['position'] == 'WR']

#lets only return relevant columns (recieving production)

WR_data = WR_data[['id', 'name', 'position', 'team', 'season', 'season_type',
         'receptions', 'targets', 'receiving_yards',
       'receiving_tds','receiving_yards_after_catch',
       'receiving_first_downs','target_share',
       'air_yards_share', 'fantasy_points', 'fantasy_points_ppr',
         'games', 'offense_snaps', 'teams_offense_snaps',
       'ypr','rec_td_percentage','rec_ypg',
         'round', 'overall', 'ht', 'wt', 'forty', 'vertical']]



Now have to merge the Stats Data with Madden ratings for each year.

Note: I originally used ratings from the Madden game associated with that season. For example, during the 2022 NFL season, Madden 23 was the latest
edition of the game. However, Madden 23 would have the rating for the player prior to the start of the 2022 season, whereas I am looking for a rating
that encapsualtes a player's performance for that season. Thus, for the 2022 NFL season, Madden 24 has ratings that are based off of 2022 NFL performance. In that sense, Madden ratings are 2 values ahead.

In [16]:
#madden 21 ratings for WRs (2019 NFL season)
madden21 = pd.read_csv('Data/maddenratings/madden21_ratings.csv')
madden21 = madden21[['Full Name', 'Overall Rating', 'Position']]
madden21WR = madden21[madden21['Position'] == 'WR']
madden21WR = madden21WR[['Full Name', 'Overall Rating']]

#add the season
madden21WR['season'] = 2019
madden21WR['season'] = madden21WR['season'].astype(int)

#renaming the 'Full Name' column to 'name' so it merge with WR_data easier
madden21WR.rename(columns = {'Full Name': 'name'}, inplace= True)

#madden21WR

In [29]:
#merging
WR_data2 = pd.merge(WR_data, madden21WR, on= ['name', 'season'], how='left')

In [30]:
#madden 23 ratings for WRs (2021 NFL season)
madden23 = pd.read_csv('Data/maddenratings/Madden23Ratings.csv')
madden23 = madden23[['fullNameForSearch','position','overall_rating']]
madden23 = madden23[madden23['position'] == 'WR']
madden23 = madden23[['fullNameForSearch','overall_rating']]

#add the season
madden23['season'] = 2021
madden23['season'] = madden23['season'].astype(int) #not sure if this is necessary but wtv

#renaming columns for merge

madden23.rename(columns = {'fullNameForSearch':'name', 'overall_rating':'Overall Rating'}, inplace=True)

In [31]:
#merging
WR_data3 = pd.merge(WR_data2, madden23, on = ['name','season'], how= 'left')

WR_data3['Overall Rating'] = np.where(WR_data3['season'] == 2019, 
                                      WR_data3['Overall Rating_x'], 
                                      np.where(WR_data3['season'] == 2021, 
                                               WR_data3['Overall Rating_y'], 
                                               np.nan))

WR_data3.drop(columns=['Overall Rating_x', 'Overall Rating_y'], inplace=True)

In [26]:
#madden 24 ratings for WRs (2022 NFL season)
madden24 = pd.read_csv('Data/maddenratings/madden24.csv')
madden24 = madden24[['Full Name', 'Position', 'Overall Rating']]
madden24 = madden24[madden24['Position'] == 'WR']
madden24 = madden24[['Full Name', 'Overall Rating']]

#add season
madden24['season'] = 2022
madden24['season'] = madden24['season'].astype(int)

#rename columns
madden24.rename(columns = {'Full Name':'name'}, inplace=True)

In [32]:
#merging
WR_data4 = pd.merge(WR_data3, madden24, on = ['name','season'], how = 'left',suffixes=('', '_2023'))

WR_data4['Overall Rating'] = np.where(WR_data4['season'] == 2022, 
                                      WR_data4['Overall Rating_2023'], 
                                      WR_data4['Overall Rating'])

WR_data4.drop(columns=['Overall Rating_2023'], inplace=True)



In [28]:
#madden 22 ratings for WRs (2020 NFL season)
madden22 = pd.read_excel('Data/maddenratings/datatime.xlsx', sheet_name = 'All Players', skiprows = 1)
madden22 = madden22[['First Name', 'Last Name', 'Position', 'Overall']]
madden22 = madden22[madden22['Position'] == 'WR']
madden22 = madden22[['First Name', 'Last Name', 'Overall']]

#create new name columm
madden22['name'] = madden22['First Name'] + ' ' + madden22['Last Name']

# add season
madden22['season'] = 2020
madden22['season'] = madden22['season'].astype(int)

#drop first name last name
madden22 = madden22.drop(columns = ['First Name', 'Last Name'])

#rename columns
madden22.rename(columns = {'Overall': 'Overall Rating'}, inplace= True)

In [33]:
#merging
WR_data_final = pd.merge(WR_data4, madden22, on= ['name','season'], how = 'left',suffixes=('', '_2022'))


WR_data_final['Overall Rating'] = np.where(WR_data_final['season'] == 2020,
                                           WR_data_final['Overall Rating_2022'],
                                           WR_data_final['Overall Rating'])

WR_data_final.drop(columns = ['Overall Rating_2022'], inplace= True)


In [36]:
#final data -> stats and madden rating for coresponding season

WR_data_final

,id,name,position,team,season,season_type,receptions,targets,receiving_yards,receiving_tds,...,ypr,rec_td_percentage,rec_ypg,round,overall,ht,wt,forty,vertical,Overall Rating
0,1,A.J. Brown,WR,PHI,2022,REG,88,145,1496,11,...,17.00,0.125,88.000000,NaN,NaN,NaN,NaN,NaN,NaN,91.0
1,1,A.J. Brown,WR,TEN,2019,REG,52,84,1051,8,...,20.21,0.154,65.687500,2.0,51.0,Jun-00,226.0,4.49,36.5,81.0
2,1,A.J. Brown,WR,TEN,2020,REG,70,106,1075,11,...,15.36,0.157,76.785714,2.0,51.0,Jun-00,226.0,4.49,36.5,86.0
3,1,A.J. Brown,WR,TEN,2021,REG,63,105,869,5,...,13.79,0.079,66.846154,2.0,51.0,Jun-00,226.0,4.49,36.5,87.0
4,4,A.J. Green,WR,ARI,2021,REG,54,92,848,3,...,15.70,0.056,53.000000,NaN,NaN,NaN,NaN,NaN,NaN,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,1767,Zay Jones,WR,BUF,2019,REG,7,18,69,0,...,9.86,0.000,17.250000,2.0,37.0,2-Jun,201.0,4.45,36.5,75.0
2396,1767,Zay Jones,WR,JAX,2022,REG,82,121,823,5,...,10.04,0.061,51.437500,NaN,NaN,NaN,NaN,NaN,NaN,81.0
2397,1767,Zay Jones,WR,LV,2019,REG,20,27,147,0,...,7.35,0.000,14.700000,NaN,NaN,NaN,NaN,NaN,NaN,75.0
2398,1767,Zay Jones,WR,LV,2020,REG,14,20,154,1,...,11.00,0.071,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,72.0


There is some data on 40 time, vertical, draft position, and height (which is misinterpreted as a date in the csv file) which I will leave incase I want to use it later but so far I don't have plans to do so.